# Problem 6.3: Classifying the Expansion of the Universe Using BAO Data
As was seen in lecture 2, the condition for which the universe's expansion is accelerating was that $w<-1/3$.

Recall that the equation of state is given as,
$$
w = P/\rho,
$$
where $P$ is the pressure, $\rho$ is the density, and $w$ which is the dimensionless equation of state parameter (here, we are working in units where $c=1$). Recall further that for matter, $w_{\rm m} = 0$ whereas for the cosmological constant, $w_\Lambda = -1$. This implies the following relations,
$$
P_\Lambda = -\rho_\Lambda, \qquad P_{\rm m} = 0.
$$
Since we are working in the late-universe, let us assume that *only* matter and the cosmological constants are contibuting components to any form of cosmic expansion, i.e. $\Omega_K \approx \Omega_R \approx 0$. This further implies that $\Omega_\Lambda = 1 - \Omega_{\rm m}$.

Now, for the *total* pressure, we can write,
$$
P_{\rm tot} = P_{\rm m} + P_\Lambda,
$$
however, using the relations above for the pressures, we can write
$$
P_{\rm tot} = P_\Lambda = -\rho_\Lambda,
$$
where we can finally use the relationship that $\rho_\Lambda = \Omega_\Lambda \rho_{\rm crit} = (1-\Omega_{\rm m}) \rho_{\rm crit}$. Therefore, we combine all of this and have,
$$
P_{\rm tot} = -(1-\Omega_{\rm m}) \rho_{\rm crit}.
$$
Next, we wish to derive some form of relationship for $\rho_{\rm tot}$,
$$
\rho_{\rm tot} = \rho_{\rm m}(z) + \rho_\Lambda.
$$
Recalling the above relationship between $\rho_\Lambda$ and $\rho_{\rm crit}$ and again from lecture 2 that $\rho_{\rm m}(z) = \Omega_{\rm m}\rho_{\rm crit}(1+z)^3$, we directly get that
$$
\rho_{\rm tot} = \Omega_{\rm m}\rho_{\rm crit}(1+z)^3 + (1-\Omega_{\rm m})\rho_{\rm crit}.
$$
Therefore, using this, the above relationship we derived for $P_{\rm tot}$ and the relation between these two and $w$, we can calculate $w_{\rm tot}$ given $\Omega_m$.

Your goal in this Colab is to create ML models which take angular diameter distances as an input and classify whether or not that data corresponds to an accelerating ($w<-1/3$) or decelerating ($w>-1/3$) universe.

a.) We will prepare data from BAO, which gives us observational values for $D_A$ as a function of Redshift. We can use `numpy`'s `np.loadtxt` function to automatically load the data from a `.txt` file and store it to an array.

In [1]:
#Import libraries
import numpy as np
import matplotlib.pyplot as plt

from astropy import units as u
from astropy.cosmology import FlatLambdaCDM  # for flat case use this

In [2]:
#The following code allows us to download the correct data files
!pip install gdown

In [3]:
#Load the data
import gdown

# Function to convert the shared link to the correct format
def convert_shared_link(shared_link):
    return shared_link.replace("?usp=share_link", "?usp=sharing")

shared_link = "https://drive.google.com/file/d/1VETN78tp7PIDpSzqUqBnSQoHArJi5P7I/view?usp=share_link"

# Convert the shared link to the correct format
shared_link_corrected = convert_shared_link(shared_link)

# Extract the file ID from the shared link
file_id = shared_link_corrected.split("/")[-2]

# Destination path to save the downloaded file in the Colab environment
destination_path = "/content/BAO_and_CMB_data.txt"  # You can modify the path as per your preference

# Download the file using gdown
gdown.download(f"https://drive.google.com/uc?id={file_id}", destination_path, quiet=False)

# Load the data
data = np.loadtxt('/content/BAO_and_CMB_data.txt')
D_A_obs = data[:,1]

D_A_obs = np.flip(data[:,1])
print(data[:,0])
print(D_A_obs)

Downloading...
From: https://drive.google.com/uc?id=1VETN78tp7PIDpSzqUqBnSQoHArJi5P7I
To: /content/BAO_and_CMB_data.txt
100%|██████████| 581/581 [00:00<00:00, 1.30MB/s]

[1.50000000e-01 3.80185957e-01 5.10522680e-01 6.99786511e-01
 8.49804744e-01 1.48071394e+00 2.29096593e+00 2.37085262e+00
 1.09000000e+03]
[  12.54019751 1634.08006572 1688.27398489 1797.73025087 1555.79854724
 1530.66619502 1308.24673246 1097.32455386  573.54982709]


In [33]:
D_A_obs

array([  12.54019751, 1634.08006572, 1688.27398489, 1797.73025087,
       1555.79854724, 1530.66619502, 1308.24673246, 1097.32455386,
        573.54982709])

b.) Let's prepare the theory data for training and testing of our models. First, we want to calculate $w$ from the above data. We will want to do so for various values of $\Omega_{\rm m}$. However, the *only* value for which this matter is the one at present day, i.e. $z=0$. So for each value of $\Omega_{\rm m}$ we need only calculate $P_{\rm tot}$ (why not $\rho_{\rm tot}$?) to calculate $w_{\rm tot}$. Then, we will use that value to create an array of labels determining if that universe's expansion is accelerating or not. On top of that, we will also want to create an array of angular diameter distances, which should go the same as in Notebook 1 of this lecture.

In [28]:
# We choose 9 points as we want to match the dimensionality of our training data to that of the BAO data
# z = np.linspace(0,50,9)
z = data[:,0]

# Generate various values of Omega_m
OmM = np.arange(0.1,0.9 + 0.01,0.01)
OmL = 1 - OmM

# Initialize empty list to append D_A and labels to
D_A = []
labels = []

check_acc = []

for i in range(len(OmM)):
  cosmo = FlatLambdaCDM(H0=67, Om0=OmM[i])
  comoving_dist = cosmo.comoving_distance(z)
  D_A.append(comoving_dist /(1+z))
  if OmL[i] > 1/3:
    check_acc.append(0)
  else: check_acc.append(1)


c.) Prepare the data for use in our ML algorithms. Make sure all labels assume values such as 0 or 1 rather than strings. Then split the data into training and testing data sets.

In [23]:
# Create a dictionary to map class labels to numerical labels
translate_dict = {'Accelerated': 0, 'Decelerated': 1}
inverse_translate_dict = {v: k for k, v in translate_dict.items()}

# Map class labels to numerical labels
numerical_labels = np.array([translate_dict[label] for label in labels])

In [29]:
X = D_A
y = check_acc

In [30]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

d.) Let's train and predict! We will use a decision tree classifier, random forest classifier, and KNN classifier.

In [31]:
from sklearn.tree import DecisionTreeClassifier
# DecisionTreeClassifier
my_tree = DecisionTreeClassifier(max_depth=4,random_state=1)

my_tree.fit(X_train, y_train)

# Print training and testing accuracies
print("Training Accuracy:",my_tree.score(X_train, y_train))
print("Testing Accuracy:",my_tree.score(X_test, y_test))

Training Accuracy: 1.0
Testing Accuracy: 0.96


In [36]:
rslt = my_tree.predict(D_A_obs.reshape(1, -1))
rslt

array([0])

In [37]:
if rslt == [0]:
  print("accelerating")
else:
  print("deccelrating")

accelerating


In [38]:
from sklearn.metrics import accuracy_score
# Evaluate the classifier
# Use model to 'predict' the test data
predictions = my_tree.predict(X_test)
# Use the accuracy metric to determine how accurate the model is
accuracy = accuracy_score(y_test, predictions)
print("Accuracy (DecisionTree):", accuracy)

Accuracy (DecisionTree): 0.96


In [39]:
from sklearn.ensemble import RandomForestClassifier
# RandomForestClassifier
my_forest = RandomForestClassifier(n_estimators=4,max_depth=4,random_state=1)
my_forest.fit(X_train, y_train)

# Print training and testing accuracies
print("Training Accuracy:",my_forest.score(X_train, y_train))
print("Testing Accuracy:",my_forest.score(X_test, y_test))

Training Accuracy: 1.0
Testing Accuracy: 0.96


In [42]:
# Evaluate the classifier
# Use model to 'predict' the test data
predictions = my_forest.predict(X_test)
# Use the accuracy metric to determine how accurate the model is
accuracy = accuracy_score(y_test, predictions)
print("Accuracy (DecisionTree):", accuracy)


Accuracy (DecisionTree): 0.96


In [43]:
# Use model to predict acceleration from BAO data
f_rslt = my_forest.predict(D_A_obs.reshape(1, -1))
if f_rslt == [0]:
  print("accelerating")
else:
  print("deccelrating")

accelerating


In [44]:
from sklearn.neighbors import KNeighborsClassifier
# KNNClassifier
# Prepare KNN with k = 5
my_knn = KNeighborsClassifier(n_neighbors=5)

# Fit KNN on standardized training data
my_knn.fit(X_train, y_train)

# Print training and testing accuracy
print("Training Accuracy:",my_knn.score(X_train, y_train))
print("Testing Accuracy:",my_knn.score(X_test, y_test))

Training Accuracy: 1.0
Testing Accuracy: 0.96


In [45]:
# Evaluate the classifier
# Use model to 'predict' the test data
predictions = my_knn.predict(X_test)
# Use the accuracy metric to determine how accurate the model is
accuracy = accuracy_score(y_test, predictions)
print("Accuracy (DecisionTree):", accuracy)

Accuracy (DecisionTree): 0.96


In [47]:
# Use model to predict acceleration from BAO data
k_rslt = my_forest.predict(D_A_obs.reshape(1, -1))
if k_rslt == [0]:
  print("accelerating")
else:
  print("deccelrating")

accelerating
